## Testing Hyper Params

In [1]:
## Librarys
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import pandas as pd
import pandas as pd
from sklearn.ensemble import RandomForestClassifier  
from sklearn.metrics import accuracy_score, confusion_matrix 
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.inspection import permutation_importance
import plotly.graph_objects as go
import plotly.express as px


In [2]:
### READ DATA ###
df = pd.read_csv('data_GEN6.csv')

# Separate target from predictors
y = df['FATAL']
X = df.drop(['FATAL', 'INJ_SEV', 'INJ_LEVEL'], axis=1)

# Check for any missing values in the dataset
if X.isnull().any().any():
    print("Warning: Missing values detected. Consider imputation or dropping rows/columns.")

# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

def get_eval(mod, X, y):
    scores = ['precision', 'accuracy', 'recall', 'f1', 'roc_auc']
    for score in scores:
        try:
            scored = cross_val_score(mod, X, y, cv=5, scoring=score)
            print(f"The model's {score} score is: {scored.mean():.4f}")
        except ValueError as e:
            print(f"Error calculating {score}: {e}")

In [3]:
# descriptive stats for the x df
X.describe()

,Alcohol Use,Age,Sex,Model Year,Height,Weight,Speed Limit,Rush Hour,Light Condition,Restraint Use,Drug Use,Cold Weather,Speeding,License Status,Airbag Deploy,Driver,Front Seat,Collision,Ejection,Large Size
count,39510.000000,39510.000000,39510.000000,39510.000000,39510.000000,39510.000000,39510.000000,39510.000000,39510.000000,39510.000000,39510.000000,39510.000000,39510.000000,39510.000000,39510.000000,39510.000000,39510.000000,39510.000000,39510.000000,39510.000000
mean,0.168616,39.896583,0.647051,2008.010326,68.299317,184.141559,50.386484,0.423462,0.698532,0.676310,0.056745,0.231081,0.174690,0.878993,0.338952,0.688610,0.870210,0.596077,0.090762,0.096608
std,0.374417,20.675239,0.477893,7.372959,3.975367,47.046856,13.295671,0.494114,0.458901,0.467889,0.231358,0.421529,0.379706,0.326140,0.473360,0.463068,0.336076,0.490689,0.287274,0.295428
min,0.000000,0.000000,0.000000,1930.000000,37.000000,58.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,23.000000,0.000000,2003.000000,66.000000,150.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,0.000000,37.000000,1.000000,2008.000000,69.000000,180.000000,55.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000
75%,0.000000,56.000000,1.000000,2014.000000,71.000000,210.000000,55.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
max,1.000000,105.000000,1.000000,2020.000000,82.000000,550.000000,80.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## RF Model with Optimized Paramaters *Random Search*

In [4]:
# Create the RandomForestRegressor model
model2 = RandomForestClassifier(n_estimators=476,
                                min_samples_split=2,
                                bootstrap=True,
                                max_depth=50,
                                n_jobs=-1,
                                min_samples_leaf = 1,
                                oob_score=True,
                                random_state = 42)

#fit model to data
model2.fit(X_train, y_train)

#make predictions
preds = model2.predict(X_valid)

#Evalueate the Accuracy
score = accuracy_score(y_valid, preds)

#feature_importance = model2.feature_importances_

# Print Results
print(f'Accuracy on validation set: {score:.4f}')  # Changed MAE to accuracy
#print(f"Validation predictions: {preds[:5]}")  # Predictions
#print(f"Actual values: {y_valid.head().tolist()}")  # Validation Y Values
#get_eval(model2, X, y)

Accuracy on validation set: 0.7936


In [8]:
from sklearn import tree
import matplotlib.pyplot as plt

clf = tree.DecisionTreeClassifier(random_state = 42)

fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(clf, 
                   feature_names=X,  
                   class_names=y,
                   filled=True)

InvalidParameterError: The 'feature_names' parameter of plot_tree must be an instance of 'list' or None. Got        Alcohol Use  Age  Sex  Model Year  Height  Weight  Speed Limit  \
0                0   34    0        2009      64     140           70   
1                0   53    1        2009      64     140           70   
2                1   59    1        2001      68     170           70   
3                0   42    0        2003      67     250           65   
4                0   54    0        2007      66     130           65   
...            ...  ...  ...         ...     ...     ...          ...   
39505            1   31    1        2010      72     175           70   
39506            1   22    1        2010      72     175           70   
39507            1   42    1        2013      67     175           70   
39508            1   41    0        2013      67     175           70   
39509            0   30    1        2013      73     290           70   

       Rush Ho

<Figure size 2500x2000 with 0 Axes>

In [ ]:
## Feature Importance ##
# generate the feature importance using permutaion importance
perm_importance = permutation_importance(model2, X_train, y_train, n_repeats=30, random_state=42)

#dictionary of perm importance
feature_importances = perm_importance['importances_mean']

#Convert importances into pandas DataFrame
features = pd.DataFrame({'feature': X_train.columns, 'importance': feature_importances})
features.sort_values(by='importance', ascending=False, inplace=True)

# Sort the DataFrame by 'Importance' in descending order
df_sorted = features.sort_values(by='importance', ascending=False)

# Creating the bar chart with sorted values
fig = go.Figure([go.Bar(x=df_sorted['feature'], y=df_sorted['importance'])])

# Updating layout for better readability
fig.update_layout(
    title='Random Forest Feature Importances',
    xaxis_tickangle=-45,
    xaxis_title='Feature',
    yaxis_title='Importance',
    template='plotly_white',
    autosize=False,
    width=1000,
    height=600
)

fig.show()

In [28]:
# Assuming y_valid are the true labels and logit_preds are the predictions from your model
conf_matrix = confusion_matrix(y_valid, preds)

# Using Plotly Express to create an interactive confusion matrix heatmap
fig = px.imshow(conf_matrix,
                labels=dict(x="Predicted labels", y="True labels", color="Number of observations"),
                x=['No Death', 'Death'],  # Adjust these based on your class labels
                y=['No Death', 'Death'],  # Adjust these based on your class labels
                text_auto=True)  # Automatically set text in each cell to the cell value

# Update layout for clarity
fig.update_layout(title=(f"Random Forest Confusion Matrix: Accuracy Score = {score:.3f}"),
                  xaxis_title="Predicted Label",
                  yaxis_title="True Label",
                  width=700,
                  height=500)

fig.show()

## RANDOM SEARCH TO FIND BEST HYPERPARAMATERS

In [5]:
### MAKE THE MODEL WITH TUNING RANDOM SEARCH###

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint

# Define the model
modelrand = RandomForestClassifier()

# Define the hyperparameters and their distributions to sample from
param_distributions = {
    'n_estimators': randint(100, 500),
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_leaf': randint(1, 9),
    'min_samples_split': randint(2, 9),
    'bootstrap': [True, False]
}

# Set up the RandomizedSearchCV
search = RandomizedSearchCV(modelrand, param_distributions=param_distributions, n_iter=100, cv=5, random_state=42, n_jobs=-1)

# Fit the search object to the data
search.fit(X_train, y_train)

# The best hyperparameters found
print("Best Hyperparameters:", search.best_params_)

# You can use the best estimator directly
best_model = search.best_estimator_

# Predictions and evaluations can then be made with this model
# Create a Pipeline to Fit the Model
my_pipeline3 = Pipeline(steps=[('model', modelrand)])
my_pipeline3.fit(X_train, y_train)
preds = my_pipeline3.predict(X_valid)

#Evalueate the Accuracy
score = accuracy_score(y_valid, preds)
feature_importance = modelrand.feature_importances_

# Print Results
print(f'Accuracy on validation set: {score:.4f}')  # Changed MAE to accuracy
print(f"Validation predictions: {preds[:5]}")  # Predictions
print(f"Actual values: {y_valid.head().tolist()}")  # Validation Y Values
#get_eval(my_pipeline, X, y)

#If you want to print feature importances:
#print(f'Feature importances: {feature_importance}')

/Users/zachcohen/anaconda3/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best Hyperparameters: {'bootstrap': True, 'max_depth': 50, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 476}
Accuracy on validation set: 0.7878
Validation predictions: [0 1 0 0 0]
Actual values: [0, 0, 0, 0, 0]


In [34]:
import pandas as pd

# Define the hyperparameters and their best values
best_hyperparameters = {
    'bootstrap': True,
    'max_depth': 50,
    'min_samples_leaf': 1,
    'min_samples_split': 2,
    'n_estimators': 476
}

# Define the accuracy
accuracy = {'Accuracy on validation set': 0.7878}

# Combine the hyperparameters and accuracy into a single dictionary
all_info = {**best_hyperparameters, **accuracy}

# Create a DataFrame
df = pd.DataFrame(list(all_info.items()), columns=['Hyperparameter', 'Value'])

df


,Hyperparameter,Value
0,bootstrap,True
1,max_depth,50
2,min_samples_leaf,1
3,min_samples_split,2
4,n_estimators,476
5,Accuracy on validation set,0.7878


In [ ]:
Hypers = pd.DataFrame({'Hyperparameter': X_train.columns, 'importance': feature_importances})


## Variable Importance

In [4]:
import plotly.graph_objects as go
import pandas as pd

# Assuming 'model' is your trained model and 'X_train' is your training dataset
feature_importance = model2.feature_importances_
features = X_train.columns

# Create a DataFrame for easy sorting
df = pd.DataFrame({'Feature': features, 'Importance': feature_importance})

# Sort the DataFrame by 'Importance' in descending order
df_sorted = df.sort_values(by='Importance', ascending=False)

# Creating the bar chart with sorted values
fig = go.Figure([go.Bar(x=df_sorted['Feature'], y=df_sorted['Importance'])])

# Updating layout for better readability
fig.update_layout(
    title='Feature Importances',
    xaxis_tickangle=-45,
    xaxis_title='Feature',
    yaxis_title='Importance',
    template='plotly_white',
    autosize=False,
    width=1000,
    height=600
)

fig.show()

In [6]:
from sklearn.inspection import permutation_importance
import plotly.graph_objects as go
import pandas as pd

# Assuming 'model' is your trained model and 'X_train' is your training dataset
MDI = permutation_importance(model2, X_train, y_train)['importances_mean']
features = X_train.columns

# Create a DataFrame for easy sorting
df = pd.DataFrame({'Feature': features, 'Importance': MDI})

# Sort the DataFrame by 'Importance' in descending order
df_sorted = df.sort_values(by='Importance', ascending=False)

# Creating the bar chart with sorted values
fig = go.Figure([go.Bar(x=df_sorted['Feature'], y=df_sorted['Importance'])])

# Updating layout for better readability
fig.update_layout(
    title='Permutation Feature Importances',
    xaxis_tickangle=-45,
    xaxis_title='Feature',
    yaxis_title='Importance',
    template='plotly_white',
    autosize=False,
    width=1000,
    height=600
)

fig.show()


In [10]:
from sklearn.inspection import permutation_importance
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd  # Make sure pandas is imported

# Assuming model2, X_train, and y_train are already defined

# Create subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=("MDI", "MDA"))

# MDI
MDI = permutation_importance(model2, X_train, y_train, n_repeats=30, random_state=0)['importances_mean']
features = X_train.columns
df = pd.DataFrame({'Feature': features, 'Importance': MDI}).sort_values(by='Importance', ascending=False)
# Add MDI trace
fig.add_trace(go.Bar(x=df['Feature'], y=df['Importance']), row=1, col=1)

# MDA
feature_importance = model2.feature_importances_
df2 = pd.DataFrame({'Feature': features, 'Importance': feature_importance}).sort_values(by='Importance', ascending=False)
# Add MDA trace
fig.add_trace(go.Bar(x=df2['Feature'], y=df2['Importance']), row=1, col=2)

# Update layout for the entire figure
fig.update_layout(title_text="Random Forest Variable Importance Scores: MDI & MDA",
                  title_x=0.5,  # Centers the title
                  xaxis_tickangle=-45,
                  xaxis_title='Feature',
                  yaxis_title='Importance',
                  template='plotly_white',
                  autosize=False,
                  width=1200,
                  height=600)

# Update axes titles
fig.update_xaxes(title_text="Feature", row=1, col=1)
fig.update_xaxes(title_text="Feature", row=1, col=2)

fig.show()


KeyboardInterrupt: 